<a href="https://colab.research.google.com/github/MANISHMOSES/Human-in-a-loop-Feedback-based-learning---Math-Routing-Agent/blob/main/Human_in_a_loop_Feedback_based_learning_Math_Routing_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/math-agent /content/drive/MyDrive/


Mounted at /content/drive
cp: cannot stat '/content/math-agent': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install chromadb > /dev/null
%pip install google-search-results > /dev/null # Also ensure serpapi is installed

In [ ]:
import os
import chromadb
from chromadb.utils import embedding_functions
from serpapi import GoogleSearch
import time
feedback_list = []
client = chromadb.Client()
collection_name = "math_knowledge_base"
collection = None

try:
    collection = client.get_collection(name=collection_name)
    print(f"Successfully connected to collection: {collection_name}")
except Exception as e:
    print(f"Error getting collection '{collection_name}'. Please ensure the KB setup step was run: {e}")

    try:
        print("Attempting to create and populate the knowledge base...")
        collection = client.create_collection(name=collection_name)

        math_data = [
            {"problem": "What is the square root of 64?", "solution": "The square root of 64 is 8 because 8 * 8 = 64."},
            {"problem": "Solve for x: 2x + 5 = 11", "solution": "Subtract 5 from both sides: 2x = 6. Divide by 2: x = 3."},
            {"problem": "What is the area of a rectangle with length 5 and width 3?", "solution": "Area = length * width. Area = 5 * 3 = 15."},
            {"problem": "What is the derivative of x^2?", "solution": "The derivative of x^2 is 2x."},
            {"problem": "What is the integral of 2x?", "solution": "The integral of 2x is x^2 + C, where C is the constant of integration."}
        ]
        ids = [f"sol_{i}" for i in range(len(math_data))]
        documents = [f"Problem: {item['problem']} Solution: {item['solution']}" for item in math_data]
        collection.add(documents=documents, ids=ids)
        print(f"Successfully created and populated collection: {collection_name}")
    except Exception as create_e:
        print(f"Failed to create and populate collection: {create_e}")
        collection = None
def input_guardrail(query: str) -> tuple[bool, str]:
    """
    Checks if the user query is related to math.
    Approach: Simple keyword matching and character checks.
    Why: This provides a basic, quick filter to ensure queries are likely math-related
         before engaging more complex downstream components. It's a first line of defense.
         It's simple to implement and understand.
    """
    math_keywords = ['math', 'solve', 'calculate', 'equation', 'formula',
                     'theorem', 'proof', 'algebra', 'geometry', 'calculus',
                     'statistics', 'probability', 'derivative', 'integral',
                     'area', 'volume', 'perimeter', 'sum', 'difference',
                     'product', 'quotient', 'find x', 'what is']

    query_lower = query.lower()

    if any(keyword in query_lower for keyword in math_keywords):
        return True, ""
    if any(char.isdigit() or char in '+-*/=^()' for char in query):
         return True, ""

    return False, "I can only answer questions related to mathematics. Please ask a math question."

def output_guardrail(solution: str) -> tuple[bool, str]:
    """
    Checks if the generated solution is math-related and appropriate for an educational context.
    Approach: Checks for presence of math-related indicators and absence of inappropriate keywords.
    Why: This acts as a final check to ensure the LLM's output is on-topic and safe before
         presenting it to the user. It helps prevent the system from generating irrelevant
         or harmful content, maintaining focus on educational math.
    """
    math_indicators = ['step', 'calculate', 'result', 'solution', 'equation',
                       'formula', 'x =', 'y =', 'answer is', 'therefore']
    inappropriate_keywords = ['violence', 'hate', 'sex', 'politics', 'unrelated topic', 'bomb', 'weapon'] # Added more inappropriate terms

    solution_lower = solution.lower()

    if any(keyword in solution_lower for keyword in inappropriate_keywords):
        return False, "The generated content contains inappropriate or off-topic material."

    if any(indicator in solution_lower for indicator in math_indicators) or \
       any(line.strip().startswith(f"{i}.") for i in range(1, 6) for line in solution.split('\n')):
        return True, ""

    if len(solution.split()) < 10 and not any(indicator in solution_lower for indicator in math_indicators):
         return False, "The generated content does not seem to be a relevant math solution."


    return True, ""
def search_knowledge_base(query: str, n_results: int = 1):
    """
    Searches the math knowledge base for relevant information.
    """
    if collection is None:
        print("Knowledge base collection not found or initialized.")
        return None

    try:

        results = collection.query(
            query_texts=[query],
            n_results=n_results
        )

        if results and results.get('documents') and results['documents'][0]:
             print(f"KB search found {len(results['documents'][0])} document(s).")
             return results
        else:
             print("KB search found no relevant documents.")
             return None
    except Exception as e:
        print(f"Error during knowledge base query: {e}")
        return None


def search_web_for_math_solution_with_mcp(query: str):
    """
    Performs a web search for a mathematical query, extracts relevant snippets,
    and formats the results according to the Model Context Protocol (MCP).
    Approach: Uses SerpAPI to perform a Google Search and extracts snippets
              from organic results, answer boxes, and knowledge graphs.
              Formats the output into a dictionary following the MCP structure
              with status and data fields.
    Why: Provides access to up-to-date information not present in the KB.
         MCP ensures a standardized format for passing web results to the LLM,
         including metadata about the source and status (success, no_results, error).
    """
    mcp_response = {
        'source': 'web_search',
        'original_query': query,
        'data': [],
        'status': 'error',
        'error_message': ''
    }

    if SERPAPI_API_KEY == "YOUR_SERPAPI_API_KEY" or not SERPAPI_API_KEY:
        mcp_response['error_message'] = "SerpAPI API key not set. Cannot perform web search."
        print(mcp_response['error_message'])
        return mcp_response

    try:
        params = {
            "q": query,
            "hl": "en",
            "gl": "us",
            "google_domain": "google.com",
            "api_key": SERPAPI_API_KEY
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        relevant_snippets = []

        if 'organic_results' in results:
            for result in results['organic_results']:
                if 'snippet' in result:
                    if any(math_term in result['snippet'].lower() for math_term in ['equation', 'formula', 'solve', 'step', 'calculate', 'proof', 'theorem', 'definition', 'property']):
                         relevant_snippets.append(result['snippet'])

        if 'answer_box' in results:
             if 'snippet' in results['answer_box']:
                 relevant_snippets.append(results['answer_box']['snippet'])
             elif 'answer' in results['answer_box']:
                  relevant_snippets.append(results['answer_box']['answer'])
             elif 'snippet_highlighted_words' in results['answer_box']:
                  # Join highlighted words if they form a coherent phrase
                  highlighted_text = " ".join(results['answer_box']['snippet_highlighted_words'])
                  if highlighted_text:
                      relevant_snippets.append(highlighted_text)

        if 'knowledge_graph' in results:
            if 'description' in results['knowledge_graph']:
                 relevant_snippets.append(results['knowledge_graph']['description'])

        if relevant_snippets:
            mcp_response['data'] = relevant_snippets
            mcp_response['status'] = 'success'
            mcp_response['error_message'] = '' # Clear error message on success
            print(f"Web search found {len(relevant_snippets)} relevant snippet(s).")
        else:
            mcp_response['status'] = 'no_results'
            mcp_response['error_message'] = "No relevant snippets found."
            print("Web search found no relevant snippets.")


    except Exception as e:
        mcp_response['status'] = 'error'
        mcp_response['error_message'] = f"Error during web search: {e}"
        print(f"Error during web search: {e}")


    return mcp_response

def route_math_query(query: str):
    """
    Routes a mathematical query to either the knowledge base or web search.
    Approach: Attempts Knowledge Base search first. If relevant results are found,
              it uses the KB. Otherwise, it falls back to Web Search.
    Why: Prioritizing the KB is generally faster and more controlled for known topics.
         Web search provides coverage for questions outside the predefined KB.
         This simple fallback mechanism ensures some information is attempted
         for most math-related queries.
    """
    print(f"\nRouting query: '{query}'")

    kb_results = search_knowledge_base(query)

    if kb_results is not None and kb_results.get('documents') and kb_results['documents'][0]:
        print("Relevant results found in knowledge base. Using knowledge base.")
        return kb_results
    else:

        print("No relevant results in knowledge base. Performing web search.")
        web_results_mcp = search_web_for_math_solution_with_mcp(query)

        return web_results_mcp


def call_llm_for_solution(prompt: str) -> str:
    """
    Placeholder function to simulate calling an LLM for a solution.
    In a real application, this would interact with an LLM API.
    """
    print("\n--- Calling Placeholder LLM ---")

    simulated_response = f"Based on the provided information:\n---\n{prompt}\n---\n\nHere is a step-by-step solution generated by the math professor:\n\n1. Understand the problem.\n2. Analyze the given information.\n3. Apply relevant mathematical concepts/formulas.\n4. Calculate or derive the solution step-by-step.\n5. State the final answer.\n\n(Note: This is a simulated LLM output. A real LLM would generate actual steps based on the context.)"
    print("--- End Placeholder LLM Call ---")
    return simulated_response


def generate_step_by_step_solution(retrieved_info: any, original_query: str) -> str:
    """
    Generates a simplified, step-by-step solution based on retrieved information.
    Approach: Takes retrieved information (either KB results or MCP web results)
              and the original query, formats them into a prompt for an LLM,
              and calls the LLM to generate the solution.
    Why: The LLM is used to synthesize information from potentially disparate sources
         (KB or web) into a coherent, simplified, step-by-step explanation as
         required by the task.
    """
    context = ""
    source_used = "Unknown"

    # Check the format of the input
    if isinstance(retrieved_info, dict) and retrieved_info.get('source') == 'web_search':
        # Input is from the web search (MCP format)
        source_used = "Web Search"
        if retrieved_info.get('status') == 'success' and retrieved_info.get('data'):
            context = "\n".join(retrieved_info['data'])
            print("Using web search results (MCP format) for solution generation.")
        elif retrieved_info.get('status') in ['no_results', 'error']:
             error_msg = retrieved_info.get('error_message', 'Unknown error during web search.')
             print(f"Web search status indicates no data or error: {retrieved_info.get('status')}. Error: {error_msg}")
             return f"Could not retrieve relevant information from web search: {error_msg}"
        else:
             print("Web search results in unexpected format or empty data.")
             return "Could not process web search results."

    elif isinstance(retrieved_info, dict) and retrieved_info.get('documents'):

        source_used = "Knowledge Base"

        if retrieved_info['documents'] and retrieved_info['documents'][0]:

            context = "\n".join(retrieved_info['documents'][0])
            print("Using knowledge base results for solution generation.")
        else:
            print("Knowledge base query returned no documents.")
            return "Could not retrieve relevant information from the knowledge base."

    else:

        print("Input information is in an unexpected format or None.")
        return "Could not process retrieved information due to unexpected format."

    if not context:
        print("No context extracted from retrieved information.")
        return "Could not generate solution: No relevant information found or extracted."


    prompt = f"""You are a helpful and patient math professor. Your task is to explain the solution to the following math problem in a clear, simplified, step-by-step manner, based *only* on the information provided below. If the provided information is not sufficient to generate a step-by-step solution, please state that you cannot provide a solution based on the available information.

Original Problem: {original_query}

Provided Information (from {source_used}):
{context}

Provide the solution steps clearly and concisely. Ensure the explanation is easy for a student to understand.
"""

    try:
        # Call the placeholder LLM function
        step_by_step_solution = call_llm_for_solution(prompt)
        return step_by_step_solution
    except Exception as e:
        # Include basic error handling for cases where the LLM call fails
        print(f"Error calling LLM: {e}")
        return f"An error occurred while generating the solution: {e}"

# --- Human-in-the-Loop Feedback Mechanism ---
def capture_feedback(query: str, solution: str, feedback: any, status: str):
    """
    Captures human feedback on a generated solution or system outcome.
    Approach: Stores relevant details (query, solution/outcome, feedback, status)
              in a global list.
    Why: Essential for the Human-in-the-Loop mechanism. It provides data
         on system performance, identifies areas for improvement (e.g.,
         incorrect solutions, guardrail issues, retrieval failures), and
         allows for evaluation and future refinement.
    """
    feedback_entry = {
        "query": query,
        "solution": solution,
        "feedback": feedback,
        "status": status,
        "timestamp": time.time()
    }
    feedback_list.append(feedback_entry)
    print("\n--- Feedback/Outcome Captured ---")
    print(f"Query: {query}")
    print(f"Status: {status}")
    print(f"Solution/Outcome (excerpt): {str(solution)[:100]}...") # Print excerpt
    print(f"Feedback: {feedback}")
    print("--- End Feedback Capture ---")

# --- Main Processing Function (Integration) ---
def process_math_query_with_guardrails_and_feedback(query: str) -> str:
    """
    Processes a user's mathematical query, including input/output guardrails,
    routing, solution generation, and feedback capture.
    Approach: Orchestrates the entire workflow:
              1. Applies input guardrail.
              2. Routes query to KB or Web.
              3. Handles retrieval outcomes.
              4. Generates solution using LLM.
              5. Applies output guardrail.
              6. Captures feedback/outcome status.
              7. Returns final response or error message.
    Why: This function serves as the central control flow, connecting all
         the individual components and ensuring they are executed in the
         correct sequence with guardrails and feedback integrated.
    """
    print(f"\n--- Processing Query: '{query}' ---")

    # 1. Apply Input Guardrail
    is_math, input_message = input_guardrail(query)
    if not is_math:
        print(f"Input Guardrail triggered: {input_message}")
        # Capture feedback for input rejection
        capture_feedback(query, input_message, None, "input_rejected")
        return input_message

    # 2. Route the query to KB or Web Search
    retrieved_info = route_math_query(query)

    # 3. Handle cases where routing or retrieval failed
    retrieval_status = "success" # Assume success initially
    retrieval_message = ""

    if isinstance(retrieved_info, dict) and retrieved_info.get('source') == 'web_search':
        if retrieved_info.get('status') in ['no_results', 'error']:
            retrieval_status = retrieved_info.get('status')
            retrieval_message = retrieved_info.get('error_message', 'Could not retrieve information from web search.')
            print(f"Retrieval failed (Web Search): {retrieval_message}")
            # Capture feedback for web retrieval failure
            capture_feedback(query, retrieval_message, retrieved_info, "retrieval_failed_web")
            return f"Could not find relevant information for your query: {retrieval_message}"
    elif retrieved_info is None or (isinstance(retrieved_info, dict) and not retrieved_info.get('documents') and not retrieved_info.get('data')):
        retrieval_status = "no_results"
        retrieval_message = "Knowledge base query returned no information."
        print(f"Retrieval failed (KB): {retrieval_message}")
        # Capture feedback for KB retrieval failure
        capture_feedback(query, retrieval_message, None, "retrieval_failed_kb")
        return "Could not find relevant information for your query."

    # 4. Generate the step-by-step solution using retrieved info
    generated_solution = generate_step_by_step_solution(retrieved_info, query)

    # 5. Handle cases where solution generation failed (e.g., LLM error, insufficient context)
    if "Could not generate solution" in generated_solution or "An error occurred while generating the solution" in generated_solution:
         print(f"Solution generation failed: {generated_solution}")
         # Capture feedback on generation failure
         capture_feedback(query, generated_solution, retrieved_info, "generation_failed")
         return generated_solution

    # 6. Apply Output Guardrail to the generated solution
    is_appropriate, output_message = output_guardrail(generated_solution)
    if not is_appropriate:
        print(f"Output Guardrail triggered: {output_message}")
        # Capture feedback when output guardrail is triggered
        capture_feedback(query, generated_solution, {"reason": output_message}, "output_rejected")
        return output_message

    # 7. If all steps succeed and guardrails pass, return the generated solution and capture feedback
    print("Guardrails passed. Returning solution and capturing feedback.")
    # Capture feedback on successful solution generation and delivery
    # In a real system, this is where actual human feedback would be solicited.
    # For this demo, we simulate positive feedback upon successful delivery.
    capture_feedback(query, generated_solution, {"placeholder_feedback": "Assume positive for now"}, "delivered")
    return generated_solution

# --- End of Combined Code ---

print("\nAll components integrated into a single code block.")
print("You can now call `process_math_query_with_guardrails_and_feedback(your_query_string)` to test the system.")


Successfully connected to collection: math_knowledge_base

All components integrated into a single code block.
You can now call `process_math_query_with_guardrails_and_feedback(your_query_string)` to test the system.


In [ ]:
# Example usage of the integrated system with a KB query
print("\n--- Testing the combined system with an example KB query ---")
test_query_kb = "What is the square root of 64?"
final_response_kb = process_math_query_with_guardrails_and_feedback(test_query_kb)
print("\nFinal Response (KB):")
print(final_response_kb)

print("\n--- Displaying captured feedback after this example ---")
display(feedback_list)


--- Testing the combined system with an example KB query ---

--- Processing Query: 'What is the square root of 64?' ---

Routing query: 'What is the square root of 64?'
KB search found 1 document(s).
Relevant results found in knowledge base. Using knowledge base.
Using knowledge base results for solution generation.

--- Calling Placeholder LLM ---
--- End Placeholder LLM Call ---
Guardrails passed. Returning solution and capturing feedback.

--- Feedback/Outcome Captured ---
Query: What is the square root of 64?
Status: delivered
Solution/Outcome (excerpt): Based on the provided information:
---
You are a helpful and patient math professor. Your task is to...
Feedback: {'placeholder_feedback': 'Assume positive for now'}
--- End Feedback Capture ---

Final Response (KB):
Based on the provided information:
---
You are a helpful and patient math professor. Your task is to explain the solution to the following math problem in a clear, simplified, step-by-step manner, based *only* on the

[{'query': 'What is the square root of 64?',
  'solution': 'Based on the provided information:\n---\nYou are a helpful and patient math professor. Your task is to explain the solution to the following math problem in a clear, simplified, step-by-step manner, based *only* on the information provided below. If the provided information is not sufficient to generate a step-by-step solution, please state that you cannot provide a solution based on the available information.\n\nOriginal Problem: What is the square root of 64?\n\nProvided Information (from Knowledge Base):\nProblem: What is the square root of 64? Solution: The square root of 64 is 8 because 8 * 8 = 64.\n\nProvide the solution steps clearly and concisely. Ensure the explanation is easy for a student to understand.\n\n---\n\nHere is a step-by-step solution generated by the math professor:\n\n1. Understand the problem.\n2. Analyze the given information.\n3. Apply relevant mathematical concepts/formulas.\n4. Calculate or derive t

In [ ]:
# Example questions for testing the integrated system

# 1. Knowledge Base Question
query_kb = "Solve for x: 2x + 5 = 11"

# 2. Web Search Question (Requires SerpAPI key to be set)
query_web = "What is the formula for the volume of a cylinder?"

# 3. Non-Math/Guardrail Question
query_non_math = "Tell me a joke."

# You can run these queries using the main processing function:
print("--- Testing Knowledge Base Query ---")
response_kb = process_math_query_with_guardrails_and_feedback(query_kb)
print("\nResponse for KB Query:")
print(response_kb)

print("\n--- Testing Web Search Query ---")
response_web = process_math_query_with_guardrails_and_feedback(query_web)
print("\nResponse for Web Search Query:")
print(response_web)

print("\n--- Testing Non-Math Query ---")
response_non_math = process_math_query_with_guardrails_and_feedback(query_non_math)
print("\nResponse for Non-Math Query:")
print(response_non_math)

# You can also display the captured feedback after running these tests
print("\n--- Displaying captured feedback after examples ---")
display(feedback_list)

--- Testing Knowledge Base Query ---

--- Processing Query: 'Solve for x: 2x + 5 = 11' ---

Routing query: 'Solve for x: 2x + 5 = 11'
KB search found 1 document(s).
Relevant results found in knowledge base. Using knowledge base.
Using knowledge base results for solution generation.

--- Calling Placeholder LLM ---
--- End Placeholder LLM Call ---
Guardrails passed. Returning solution and capturing feedback.

--- Feedback/Outcome Captured ---
Query: Solve for x: 2x + 5 = 11
Status: delivered
Solution/Outcome (excerpt): Based on the provided information:
---
You are a helpful and patient math professor. Your task is to...
Feedback: {'placeholder_feedback': 'Assume positive for now'}
--- End Feedback Capture ---

Response for KB Query:
Based on the provided information:
---
You are a helpful and patient math professor. Your task is to explain the solution to the following math problem in a clear, simplified, step-by-step manner, based *only* on the information provided below. If the provi

[{'query': 'Solve for x: 2x + 5 = 11',
  'solution': 'Based on the provided information:\n---\nYou are a helpful and patient math professor. Your task is to explain the solution to the following math problem in a clear, simplified, step-by-step manner, based *only* on the information provided below. If the provided information is not sufficient to generate a step-by-step solution, please state that you cannot provide a solution based on the available information.\n\nOriginal Problem: Solve for x: 2x + 5 = 11\n\nProvided Information (from Knowledge Base):\nProblem: Solve for x: 2x + 5 = 11 Solution: Subtract 5 from both sides: 2x = 6. Divide by 2: x = 3.\n\nProvide the solution steps clearly and concisely. Ensure the explanation is easy for a student to understand.\n\n---\n\nHere is a step-by-step solution generated by the math professor:\n\n1. Understand the problem.\n2. Analyze the given information.\n3. Apply relevant mathematical concepts/formulas.\n4. Calculate or derive the soluti

In [ ]:
# Example math questions
example_queries = [
    "What is the area of a circle with radius 7?",
    "Solve the inequality: 3x - 2 > 10",
    "What is the definite integral of x^2 from 0 to 1?"
]

for query in example_queries:
    print(f"- {query}")

- What is the area of a circle with radius 7?
- Solve the inequality: 3x - 2 > 10
- What is the definite integral of x^2 from 0 to 1?
